In [1]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [2]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

In [3]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exchange'})

In [4]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [5]:
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
nyse_companies = all_companies[(all_companies.exchangeShortName == 'NYSE')]
nasdaq_companies = all_companies[(all_companies.exchangeShortName == 'NASDAQ')]

lse_stocks = lse_companies[(lse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nyse_stocks = nyse_companies[(nyse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nasdaq_stocks = nasdaq_companies[(nasdaq_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

print(f"num LSE companies: {len(lse_companies)}")
print(f"num LSE stocks: {len(lse_stocks)}")
print()
print(f"num NYSE companies: {len(nyse_companies)}")
print(f"num NYSE stocks: {len(nyse_stocks)}")
print()
print(f"num NASDAQ companies: {len(nasdaq_companies)}")
print(f"num NASDAQ stocks: {len(nasdaq_stocks)}")

num LSE companies: 6335
num LSE stocks: 3877

num NYSE companies: 4070
num NYSE stocks: 3463

num NASDAQ companies: 10249
num NASDAQ stocks: 5916


In [7]:
thread_lock = threading.Lock()

def get_exchange_data(exchange: str):
    datestamp = datetime.now().strftime('%y%m%d')
    exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
    exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

    symbols = exchange_stocks.symbol.tolist()
    stock_data = {}
    failed = []
    num_stocks = len(symbols)
    cnt = 0

    symbol_queue = queue.Queue()
    [symbol_queue.put(s) for s in symbols]
    symbol_queue.qsize()

    def worker():
        nonlocal cnt
        nonlocal stock_data
        nonlocal failed
        while True:
            with thread_lock:
                cnt += 1
                symbol = symbol_queue.get()        
                print(f'{exchange} [{cnt}/{num_stocks}]: getting {symbol} - f[{len(failed)}], s[{len(stock_data)}]')

            if symbol in stock_data:
                print(f"Already have {symbol}, skipping")
                continue

            try:
                data = fa_utils.get_ticker_data(symbol, api_key)
                stock_data[symbol] = data
                print(f"{symbol} SUCCESS")
            except:
                print(f"{symbol} FAILED")
                failed.append(symbol)

            symbol_queue.task_done()

    for x in range(4):
        name = "Thread_" + str(x)
        t = threading.Thread(name=name,target=worker, daemon=True).start()

    symbol_queue.join()

    print(len(failed))
    print(len(stock_data))

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
        pickle.dump(stock_data, file)

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
        pickle.dump(failed, file)

    return stock_data, failed

In [8]:
exchange = 'NASDAQ'
data, failed = get_exchange_data(exchange)

NASDAQ [1/5916]: getting TWNK - f[0], s[0]
NASDAQ [2/5916]: getting SIER - f[0], s[0]
NASDAQ [3/5916]: getting SMIHW - f[0], s[0]
NASDAQ [4/5916]: getting HYACW - f[0], s[0]
SMIHW FAILEDHYACW FAILED
NASDAQ [5/5916]: getting RAVE - f[1], s[0]

NASDAQ [6/5916]: getting TUSK - f[2], s[0]
SIER SUCCESS
NASDAQ [7/5916]: getting OFSSH - f[2], s[1]
TWNK SUCCESS
NASDAQ [8/5916]: getting UONEK - f[2], s[2]
TUSK SUCCESS
NASDAQ [9/5916]: getting UNAM - f[2], s[3]
OFSSH FAILED
NASDAQ [10/5916]: getting HMCOU - f[3], s[3]
RAVE SUCCESS
NASDAQ [11/5916]: getting IMTE - f[3], s[4]
IMTE FAILED
NASDAQ [12/5916]: getting GAINZ - f[4], s[4]
HMCOU SUCCESS
NASDAQ [13/5916]: getting UCBIO - f[4], s[5]
UONEK SUCCESS
NASDAQ [14/5916]: getting SPKBW - f[4], s[6]
GAINZ FAILED
NASDAQ [15/5916]: getting GLSPW - f[5], s[6]
SPKBW FAILED
NASDAQ [16/5916]: getting LDHAW - f[6], s[6]
UNAM SUCCESS
NASDAQ [17/5916]: getting SANM - f[6], s[7]
GLSPW FAILED
NASDAQ [18/5916]: getting QRTEA - f[7], s[7]
LDHAW FAILED
NASDAQ [19

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'rb') as file:
    stock_data_2 = pickle.load(file)

print(len(stock_data_2))
display(stock_data_2)